In [1]:
# 1. 데이터 확보
# 압축 풀기
import zipfile
filepath = 'data/glaucoma.zip'
with zipfile.ZipFile(filepath, 'r') as f:
    f.extractall('data/')

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(
                      rotation_range = 20,     # 최대 회전 각도
                      width_shift_range = .2,  # 가로 방향 이동, 가로 크기의 -20% ~ 20% 범위 내
                      height_shift_range = .2, # 세로 방향 이동, 세로 크기의 -20% ~ 20% 범위 내
                      horizontal_flip = True,  # 수평 뒤집기
                      vertical_flip   = True,  # 수직 뒤집기
                  )
train_data = train_generator.flow_from_directory(
    'data/glaucoma/train',
    target_size = (224, 224),
    batch_size  = 4, 
    class_mode  = 'sparse', # 종속변수에 One-Hot Encoding 안 함
)
test_generator = ImageDataGenerator()
test_data = test_generator.flow_from_directory(
    'data/glaucoma/test',
    target_size = (224, 224),
    batch_size  = 4, 
    class_mode  = 'sparse', # 종속변수에 One-Hot Encoding 안 함
)

Found 828 images belonging to 3 classes.
Found 320 images belonging to 3 classes.


In [20]:
# Train / Test Dataset이 될 Image의 channel 확인
# image를 Numpy Array로 가져오기
img = image.load_img('data/glaucoma/test/normal_control/1.png') # Image 타입
img_data = image.img_to_array(img) # Numpy로 변환
img_data.shape

(240, 240, 3)

In [2]:
# 2. Model 생성
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import decode_predictions
import numpy as np
from tensorflow.keras.models import Sequential # ResNet50의 Dense층을 사용자 추가
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing import image # 이미지 읽어오기

resnet_model = ResNet50(
                    input_shape = (224, 224, 3),
                    include_top = False) # Convolution과 Pooling Layer의 구조와 가중치를 가져옴
resnet_model.trainable = True # ResNet의 가중치도 update 가능
# model = Sequential()
# model.add(resnet_model)
# model.add(Flatten())
# model.add(Dense(units = 1024, activation = 'relu'))
# model.add(Dense(units = 3, activation = 'softmax'))
model = Sequential([
    resnet_model,
    Flatten(),
    Dense(units = 1024, activation = 'relu'),
    Dense(units = 3, activation = 'softmax')
])
model.summary()
# 학습 시 필요한 용량
    # 1. Parameter Memory Usage: params(126,352,259개)*4byte(float) / 1024^2 = 482MB
    # 2. batch_size = 3일 경우, 3*(224 x 224 x 3)*4byte / 1024^2 = 19MB
    # 3. 각 layer층마다 활성화 값

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 1024)              102761472 
                                                                 
 dense_1 (Dense)             (None, 3)                 3075      
                                                                 
Total params: 126,352,259
Trainable params: 126,299,139
Non-trainable params: 53,120
_________________________________________________________________


In [4]:
# 3. Training Process Setting
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy']) # recall, precision
# 4. Training
model.fit(train_data, 
          validation_data = test_data,
          epochs = 20, 
          batch_size = 1)

Epoch 1/20


ResourceExhaustedError: in user code:

    File "C:\Users\4545\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\4545\anaconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\4545\anaconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\4545\anaconda3\lib\site-packages\keras\engine\training.py", line 997, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "C:\Users\4545\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\optimizer_v2.py", line 579, in minimize
        return self.apply_gradients(grads_and_vars, name=name)
    File "C:\Users\4545\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\optimizer_v2.py", line 695, in